In [1]:
## Construct LSTM using Tensorflow + Keras
# Import Libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras import optimizers

#Import the helper functions
#from helperFunctions import CSP
from helperFunctions import GetCombinedData_CLA as GetCLA


enableDropout = True;
dropoutPercentage = 0.3;

#Run GetCombinedData to pull the datasets from multiple subjects into a single set
Data, Targets, DataCSP, TargetsCSP = GetCLA('../../../matDown/CLA_Data', True);

## Split into train and test sets
DataTrain, DataTest, TargetsTrain, TargetsTest = train_test_split(Data, Targets, test_size=0.3, random_state=0)

## Reshape the data for time-series processing
## Syntax np.reshape((numExamples, numTimeSteps, numInputs/numFeatures))
DataTrainRe = DataTrain.reshape((DataTrain.shape[0], DataTrain.shape[1], DataTrain.shape[2]))
DataTestRe = DataTest.reshape((DataTest.shape[0], DataTest.shape[1], DataTest.shape[2]))


Processing dataset 1 of 14
Processing dataset 2 of 14
Processing dataset 3 of 14
Processing dataset 4 of 14
Processing dataset 5 of 14
Processing dataset 6 of 14
Processing dataset 7 of 14
Processing dataset 8 of 14
Processing dataset 9 of 14
Processing dataset 10 of 14
Processing dataset 11 of 14
Processing dataset 12 of 14
Processing dataset 13 of 14
Processing dataset 14 of 14


/Users/daniel/Documents/GitHub/ECE228ECGProject/JupyterNotebooks/helperFunctions.py:83: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


In [2]:

## Construct the model
LSTM_EEG = Sequential()
LSTM_EEG.add(LSTM((100),batch_input_shape=(None,DataTrainRe.shape[1], DataTrainRe.shape[2]), return_sequences=True))
if(enableDropout):
    LSTM_EEG.add(Dropout(dropoutPercentage))
LSTM_EEG.add(LSTM((50), return_sequences=False))
if(enableDropout):
    LSTM_EEG.add(Dropout(dropoutPercentage))
LSTM_EEG.add(Dense((2),activation='sigmoid'))

LSTM_EEG.summary()
sgd = optimizers.SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=True)
LSTM_EEG.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

history = LSTM_EEG.fit(DataTrain, TargetsTrain, epochs=30,verbose=2, batch_size=16)

predictionsTest = LSTM_EEG.predict(DataTest)

predictionsTest[predictionsTest>0.5] = 1
predictionsTest[predictionsTest <= 0.5] = 0
comparisonArrayTest = predictionsTest == TargetsTest

correctCountTest = 0
for boolValues in comparisonArrayTest:
    if(boolValues[0] & boolValues[1]):
        correctCountTest += 1
falseCountTest = DataTest.shape[0] - correctCountTest

predictionsTrain = LSTM_EEG.predict(DataTrain)
predictionsTrain[predictionsTrain>0.5] = 1;
predictionsTrain[predictionsTrain<=0.5] = 0;
comparisonArrayTrain = predictionsTrain == TargetsTrain;

correctCountTrain = 0
for boolValues in comparisonArrayTrain:
    if(boolValues[0] & boolValues[1]):
        correctCountTrain += 1
falseCountTrain = DataTrain.shape[0] - correctCountTrain


# In[87]:
trainAcc_noCSP = (correctCountTrain*100/DataTrain.shape[0]);
testAcc_noCSP = (correctCountTest*100/DataTest.shape[0]);

print("#################################")
print("#################################")
print("Training Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTrain, falseCountTrain, trainAcc_noCSP))
print("#################################")
print("#################################")
print("Testing Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTest, falseCountTest, testAcc_noCSP))
print("#################################")
print("#################################")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 21, 100)           120800    
_________________________________________________________________
dropout (Dropout)            (None, 21, 100)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense (Dense)                (None, 2)                 102       
Total params: 151,102
Trainable params: 151,102
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
370/370 - 5s - loss: 0.6963 - accuracy: 0.5263
Epoch 2/30
370/370 - 6s - loss: 0.6845 - accuracy: 0.5557


# Results without CSP

### Training Performance:
Correct MI Prediction: 4820

Incorrect MI Prediction: 1099

Percent Accuracy: 81.433%
#################################


### Testing Performance:
Correct MI Prediction: 1467

Incorrect MI Prediction: 1070

Percent Accuracy: 57.824%
#################################


We can see that training the LSTM on the combined data reduces overfitting but still provides only mediocre test results.

In the next cell, we train the same model on the CSP data, which has already been generated

In [3]:
## Split into train and test sets
DataTrain, DataTest, TargetsTrain, TargetsTest = train_test_split(DataCSP, TargetsCSP, test_size=0.3, random_state=0)

## Reshape the data for time-series processing
## Syntax np.reshape((numExamples, numTimeSteps, numInputs/numFeatures))
DataTrainRe = DataTrain.reshape((DataTrain.shape[0], DataTrain.shape[1], DataTrain.shape[2]))
DataTestRe = DataTest.reshape((DataTest.shape[0], DataTest.shape[1], DataTest.shape[2]))

history = LSTM_EEG.fit(DataTrain, TargetsTrain, epochs=30,verbose=2, batch_size=16)

predictionsTest = LSTM_EEG.predict(DataTest)

predictionsTest[predictionsTest>0.5] = 1
predictionsTest[predictionsTest <= 0.5] = 0
comparisonArrayTest = predictionsTest == TargetsTest

correctCountTest = 0
for boolValues in comparisonArrayTest:
    if(boolValues[0] & boolValues[1]):
        correctCountTest += 1
falseCountTest = DataTest.shape[0] - correctCountTest

predictionsTrain = LSTM_EEG.predict(DataTrain)
predictionsTrain[predictionsTrain>0.5] = 1;
predictionsTrain[predictionsTrain<=0.5] = 0;
comparisonArrayTrain = predictionsTrain == TargetsTrain;

correctCountTrain = 0
for boolValues in comparisonArrayTrain:
    if(boolValues[0] & boolValues[1]):
        correctCountTrain += 1
falseCountTrain = DataTrain.shape[0] - correctCountTrain


# In[87]:
trainAcc_noCSP = (correctCountTrain*100/DataTrain.shape[0]);
testAcc_noCSP = (correctCountTest*100/DataTest.shape[0]);

print("#################################")
print("#################################")
print("Training Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTrain, falseCountTrain, trainAcc_noCSP))
print("#################################")
print("#################################")
print("Testing Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTest, falseCountTest, testAcc_noCSP))
print("#################################")
print("#################################")

Epoch 1/30
370/370 - 6s - loss: 0.6724 - accuracy: 0.5898
Epoch 2/30
370/370 - 6s - loss: 0.5742 - accuracy: 0.6930
Epoch 3/30
370/370 - 6s - loss: 0.5185 - accuracy: 0.7417
Epoch 4/30
370/370 - 6s - loss: 0.4833 - accuracy: 0.7640
Epoch 5/30
370/370 - 6s - loss: 0.4712 - accuracy: 0.7748
Epoch 6/30
370/370 - 6s - loss: 0.4399 - accuracy: 0.7927
Epoch 7/30
370/370 - 6s - loss: 0.4103 - accuracy: 0.8081
Epoch 8/30
370/370 - 6s - loss: 0.4103 - accuracy: 0.8128
Epoch 9/30
370/370 - 6s - loss: 0.3860 - accuracy: 0.8255
Epoch 10/30
370/370 - 7s - loss: 0.3668 - accuracy: 0.8375
Epoch 11/30
370/370 - 7s - loss: 0.3490 - accuracy: 0.8485
Epoch 12/30
370/370 - 6s - loss: 0.3391 - accuracy: 0.8557
Epoch 13/30
370/370 - 6s - loss: 0.3219 - accuracy: 0.8623
Epoch 14/30
370/370 - 6s - loss: 0.3203 - accuracy: 0.8625
Epoch 15/30
370/370 - 6s - loss: 0.2959 - accuracy: 0.8773
Epoch 16/30
370/370 - 6s - loss: 0.2969 - accuracy: 0.8714
Epoch 17/30
370/370 - 7s - loss: 0.3009 - accuracy: 0.8719
Epoch 

# Results with CSP
---
### Training Performance:
Correct MI Prediction: 5638
    
Incorrect MI Prediction: 281
    
Percent Accuracy: 95.253%

---

### Testing Performance:
Testing Performance:
    
Correct MI Prediction: 2266
    
Incorrect MI Prediction: 271
    
Percent Accuracy: 89.318%

---
We see that CSP greatly increases the accuracy for both the training and testing sets.